In [2]:
import numpy
import nibabel as nib
import os
import matplotlib.pyplot as plt
# Importing our own functions
from functions.Volume import Volume
from functions.label import SegmentationLabel

# Example running the code
In this notebook will guide on how to use a segmentation file and obtain MR property volumes from it

In [3]:
# Inside the example folder we have already segmented files from CT
# dataset by Gatidis et al. https://doi.org/10.1038/s41597-022-01718-3
# Attribution to the data set link : https://creativecommons.org/licenses/by/4.0/
ct_wb_seg = nib.load("example/final_total_seg.nii.gz") # wb stands for whole body
# We can acquire the numpy data from the nifti
labels = ct_wb_seg.get_fdata()
# And we can quickly see the shape of our data
labels.shape

(400, 400, 326)

# MR Property Phantom Creation

In [4]:
# Here we call the Volume function
# This volume function works for whole body - Total Seg CT
new_vol = Volume(ct_wb_seg)

In [5]:
# Given that we know that all the labels from Total Seg should be the same
# We can set all the labels automatically with a specific method
new_vol.create_segmentation_labels()

Label ID 256 not found.
Label ID 256 not found.
Label ID 289 not found.
Label ID 289 not found.


This Labels not found are pre-defined labels that work with final_sc_seg.nii.gz </br>
This file will be used later in the example of creating new labels :)

In [6]:
%%time
# Now we have all the labels from the whole body stablished
# By default T2star and Susceptibility values are complete 
# You can check the values used in the look-up table on the README file
sus_dist = new_vol.create_sus_dist()
# The output will be a Numpy array 

CPU times: total: 3.55 s
Wall time: 16.9 s


In [7]:
# A method is created to save the susceptibility from Numpy array to Nifti file
new_vol.save_sus_dist_nii()
# By default it will save it in the "output" folder

In a similar way we can create proton density and T2star volumes, acquire their numpy type data and then save them as nifti files

In [8]:
%%time
pd_vol = new_vol.create_pd_vol()
t2_star_vol = new_vol.create_t2_star_vol()

CPU times: total: 5.05 s
Wall time: 31.9 s


In [9]:
new_vol.save_pd_dist()
new_vol.save_t2star_dist()

Now you will be able to open any of this 3 volumes on a viewer of choice. </br>
As mentioned in the README file, if you use ITK snap you can open the segmentation labels.

# Label creation

This section will guide you through how to create new labels, you will need SCT toolbox: https://spinalcordtoolbox.com/index.html by De Leener B, Levy S, Dupont SM, Fonov VS, Stikov N, Louis Collins D, Callot V, Cohen-Adad J. SCT: Spinal Cord Toolbox, an open-source software for processing spinal cord MRI data. Neuroimage 2017. </br>
In this example we will address a current limitation from Total Seg CT, that is that we cannot differentiate CSF, WM or GM from the Spinal Cord. </br>
In order to add them we need to first register the CT image to the PAM50 space. For this we can use the Spinal Cord segmentation and you can manually create landmarks of spine C2 and C5. With this files ready you can automatically register to PAM50 space and procede with creating new labels. </br> 
If you want, you can also manually add a label (through a process of your liking) and also follow through the steps below. </br>
Given that the labeling from anatomical image will have a specific value, when we add a new label we must add it with a value that is not included in the look up table. For example, for CT whole body Total Seg we must use a number higher than 117 to avoid conflicts.

In [ ]:
# The trick to add a new semgnetation is to use the mask and invert its values
sct_maths -i new_label.nii.gz -o inv.nii.gz -sub 1
sct_maths -i inv.nii.gz -o inv.nii.gz -mul -1
sct_maths -i temp.nii.gz -o no_label.nii.gz -mul inv.nii.gz

The label we want to add is new_label and we invert it. Then, as we dont want an overlap of labels we multiply the the inversed segmentation and multiply with the volume we want to add the label. Note that we use temp as it is better to create a copy and edit the copy instead of the original.

In [10]:
#Now the body will have zeroes where the label is going to go
# We need to set the value of the label to a number that 
# does not interfere with the look-up table
# For this example, any number higher than 117 would work 
sct_maths -i reslice_label.nii.gz -o new_label.nii.gz -mul 16
sct_maths -i new_label.nii.gz -o new_label.nii.gz -mul 16

We choose to multiply twice because there is a maximum value of 40 to multiply in SCT_maths. We can then create a new nifti file with value of 256 for the mask and 0 elsewhere. </br>
Then we just need to add the new label file to the no_label file

In [ ]:
sct_maths -i no_label.nii.gz -o pro_labels.nii.gz -add new_label.nii.gz

## Important notes
- Once a new label is created, if you want to add more labels you will have to work on top of any previously made new volume with added labels, so creating a temporal copy to test first is recommended. </br>
- Also recommended, check the mask before multplication as I have deleted some files by multiplying inappropriate inversed masks. </br>
- In the "example" folder you will find an example of a succesfull implementation of 2 new masks to the CT_wb data that adds labels 289 and 256 to the whole body dataset.